In [9]:
%matplotlib inline

# Data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Torch
import torch
import torch.nn as nn

# General
import glob
from tqdm.notebook import tqdm

# Dataset loading

In [12]:
# Key stats dataframe
key_stats_df = pd.read_csv("data/key_stats_yahoo.csv")

# Prices dataframes for every stock
prices_files = glob.glob("data/prices/*.csv")
prices_dfs = [pd.read_csv(f) for f in tqdm(prices_files)]

  0%|          | 0/429 [00:00<?, ?it/s]

In [15]:
print("Key stats dataframe format:")
print(key_stats_df.head())

print("Prices dataframe format:")
print(prices_dfs[0].head())

Key stats dataframe format:
   Unnamed: 0              Date Ticker    Price  DE Ratio  Trailing P/E  \
0           0   1/30/2004 19:01      a  33.8320     0.407           NaN   
1           1    4/13/2004 4:07      a  28.4626     0.375           NaN   
2           2  12/13/2004 21:37      a  21.8816     0.323         33.58   
3           3    1/22/2005 5:17      a  20.2478     0.322         30.70   
4           4     1/9/2006 2:35      a  31.7118     2.169         52.82   

   Price/Sales  Price/Book  Profit Margin  Operating Margin  ...  Diluted EPS  \
0         2.80        6.01          29.56             11.97  ...        3.774   
1         2.43        4.98          25.56              6.20  ...        3.391   
2         1.61        3.24           4.86              5.38  ...        0.710   
3         1.52        3.04           4.86              5.38  ...        0.710   
4         3.31        4.26           6.38              3.58  ...        0.660   

   Earnings Growth  Revenue Growth